# Imports

In [1]:
!pip install -qq tensorflow-gpu==1.14 keras tqdm

     |████████████████████████████████| 377.0MB 42kB/s 
     |████████████████████████████████| 491kB 45.2MB/s 
     |████████████████████████████████| 3.2MB 53.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.


In [0]:
import json
import csv
import pandas as pd
import numpy as np
from math import floor, ceil, sin, cos, radians, asin, sqrt, atan2

import itertools
import pickle

import matplotlib.pyplot as plt

from copy import deepcopy
from tqdm import tqdm
from pathlib import Path

# Functions

In [0]:
def get_RSS_list(gsm_ncells_str):
  """
  Parses the list of GSM_NCELLS read from the csv file 
  """
  gsm_ncells_str = gsm_ncells_str.replace('] [', '][').replace(']', '],').replace(' , ', ',').replace('[ ', '[').replace(' ]', ']')[:-2]
  gsm_ncells_str = '[' + gsm_ncells_str + ']'
  return json.loads(gsm_ncells_str)

def change_datatypes(list_dicts):
  """
  Changes the datatypes of some of the variables
  GSM_CID:    changed to int
  GSM_LAC:    changed to int
  GSM_RSS:    changed to int and converted to dBm
  GSM_NCELLS: each RSS reading is changed to dBm, and the primary cell is added
              as the first element to this list 
  """
  list_dicts_ = deepcopy(list_dicts)

  for i, dic in enumerate(list_dicts_):
    dic['GSM_CID'] = int(dic['GSM_CID'])
    dic['GSM_LAC'] = int(dic['GSM_LAC'])
    dic['GSM_RSS'] = -113 + 2 * int(dic['GSM_RSS'])
    if dic['GSM_RSS'] == 85:
      dic['GSM_RSS'] = 0
    dic['GSM_NCELLS'] = get_RSS_list(dic['GSM_NCELLS'])
    dic['accuracy'] = float(dic['accuracy'])
    dic['latitude'] = float(dic['latitude'])
    dic['longitude'] = float(dic['longitude'])
    
    lst = []
    for elem in dic['GSM_NCELLS']:

      if elem[0] == dic['GSM_CID']:
          continue

      elem[2] = -113 + 2 * elem[2]
      if elem[2] == 85:         ## Unknown RSS, skip
        continue

      lst.append(elem[:3])
    lst.insert(0, [dic['GSM_CID'], dic['GSM_LAC'], dic['GSM_RSS']])

    dic['GSM_NCELLS'] = lst
  return list_dicts_

def get_unique_cells(list_dicts):
  """
  Gets a list of all the unique cells in the dataset
  """
  unique_cells = []
  for elem in list_dicts:
    for lst in elem['GSM_NCELLS']:
      if lst[0] not in unique_cells:
        unique_cells.append(lst[0])
  return unique_cells

def get_cell_indxs(list_dicts=None, cells=None):
  """
  Creates a mapping from cell number to it's index in the input layer
  """
  if cells is None:
    cells = get_unique_cells(list_dicts)
    
  cells = sorted(cells)

  cell_indxs = {}
  for i, cell in enumerate(cells):
    cell_indxs[cell] = i

  return cell_indxs

def combine_cells(list_dicts, cell_indxs=None, zero_value=-108):
  """
  Takes the list of dictionaries and coverts the all the RSS readings of each row 
  to one list using the cell_indxs mapping
  """
  if cell_indxs is None:
      cell_indxs = get_cell_indxs(list_dicts)
  list_dicts_ = []
  for i,row in enumerate(deepcopy(list_dicts)):
    GSM_NCELLS = np.zeros(len(cell_indxs.keys()), dtype='int')
    for cell in row['GSM_NCELLS']:
      try:
        if GSM_NCELLS[cell_indxs[cell[0]]] != 0:
          print(i, row)
        GSM_NCELLS[cell_indxs[cell[0]]] = cell[2]
      except:
        pass
    
    row['CELLS_COUNT'] = np.count_nonzero(GSM_NCELLS)
    if row['CELLS_COUNT'] == 0:
      continue
    GSM_NCELLS[GSM_NCELLS == 0] = zero_value
    row['GSM_NCELLS'] = GSM_NCELLS
    
    
    list_dicts_.append(row)
  return list_dicts_


#==================================================LABELS=============================================
def get_min_max_coords(list_dicts):
  """
  Takes the list of dictionaries and get the minimum and maximum longitude and latitude
  """
  min_lat, min_lon, max_lat, max_lon = 500, 500, -500, -500

  for dct in list_dicts:
    lat = dct['latitude']
    lon = dct['longitude']
    
    if lat < min_lat:
      min_lat = lat
    if lat > max_lat:
      max_lat = lat
      
    if lon < min_lon:
      min_lon = lon
    if lon > max_lon:
      max_lon = lon
    
  return (min_lat, max_lat), (min_lon, max_lon)


def get_distance(coords1, coords2):
  lat1, lon1, lat2, lon2 = map(radians, [coords1[0], coords1[1], coords2[0], coords2[1]]) 
  dlon = radians(coords2[1] - coords1[1]) 
  dlat = radians(coords2[0] - coords1[0])
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  km = 6371* c
  return km*1000

def get_gridcell2(coords, min_max_coords, num_rows, num_cols, grid_cell_length=100):
  lat_sw, lon_sw, lat_ne, lon_ne = min_max_coords
  lon_nw, lon_se = lon_sw, lon_ne
  lat_se, lat_nw = lat_sw, lat_ne
  
  se = (lat_se, lon_se)
  sw = (lat_sw, lon_sw)
  ne = (lat_ne, lon_ne)
  nw = (lat_nw, lon_nw)

  lat_test, lon_test = coords
  
  bottom = (lat_sw, lon_test)
  left = (lat_test, lon_nw)
  top=(lat_nw, lon_test)
  
  row_dist, col_dist = get_distance(left, sw), get_distance(bottom, sw)
  row = int(row_dist // grid_cell_length)
  col = int(col_dist // grid_cell_length)
  
  cell = row * num_cols + col
  
  return cell
  
def assign_cells_to_data_2(list_dicts, min_max_coords, grid_cell_length=100):  
  lat_sw, lon_sw, lat_ne, lon_ne = min_max_coords
  lat_se, lon_se = lat_sw, lon_ne
  lat_nw, lon_nw = lat_ne, lon_sw
  
  se = (lat_se, lon_se)
  sw = (lat_sw, lon_sw)
  ne = (lat_ne, lon_ne)
  nw = (lat_nw, lon_nw)
  
  ver_distance1 = get_distance(sw, nw)
  ver_distance2 = get_distance(se, ne)

  hor_distance1 = get_distance(sw, se)
  hor_distance2 = get_distance(nw, ne)

  avg_hor_distance = (hor_distance1 + hor_distance2) / 2
  avg_ver_distance = (ver_distance1 + ver_distance2) / 2

  num_cols = np.ceil(avg_hor_distance / grid_cell_length)
  num_rows = np.ceil(avg_ver_distance / grid_cell_length)
  
  
  num_rows = int(num_rows)
  num_cols = int(num_cols)
  for row in list_dicts:
    row['label'] = get_gridcell2((row['latitude'], row['longitude']), min_max_coords, num_rows, num_cols, grid_cell_length=grid_cell_length)
  return num_rows, num_cols
#==========================================================================================
def create_data_matrix(list_dicts, output_len, get_onehot=True):
  """
  Takes a list of dictionaries as input and return two matrices, one for
  the data and one for the labels (id of the grid cells)
  """
  number_gsm_cells = list_dicts[0]['GSM_NCELLS'].shape[0]
  x = np.empty((list_dicts.shape[0], number_gsm_cells), dtype='int')
  y = np.empty((list_dicts.shape[0], output_len), dtype='uint8')
  y_nums = np.empty((len(list_dicts), )).astype('uint32')
 
  for i, elem in enumerate(list_dicts):
    x[i] = elem['GSM_NCELLS'].reshape(1, -1)
    if get_onehot:
      y_one_hot = np.zeros(output_len, dtype='int').reshape(1, -1)[0]
      y_one_hot[elem['label']] = 1
      y[i] = y_one_hot
    else:
      y[i] = 0
    y_nums[i] = elem['label']
  y_coords = get_coords_labels(list_dicts)
  timestamps = get_timestamps(list_dicts)
  return (x, y, y_nums, y_coords, timestamps)

def get_all_key_elements_from_dicts(dicts, key):
  num_rows, num_cols = len(dicts), dicts[0][key].shape[0]
  data = np.empty((num_rows, num_cols))
  for i, dic in enumerate(dicts):
    data[i] = dic[key]
    
  return data

def get_unique_dicts(dicts, key):
  all_rss = get_all_key_elements_from_dicts(dicts, key)
  _, idxs = np.unique(all_rss, axis=0, return_index=True)
  
  unique_dicts = []
  for idx in idxs:
    unique_dicts.append(dicts[idx])
    
  return unique_dicts

def train_val_test_split(list_dicts, val_ratio=0.2, test_ratio=0.2, shuffle=True):
  """
  Performs train-val-test split according to the given ratios
  """
  train_ratio = 1 - val_ratio - test_ratio
  number_elements = len(list_dicts)
  idxs = np.arange(number_elements)
  if shuffle:
    np.random.shuffle(idxs)
  list_dicts = np.array(list_dicts)[idxs]
  
  train_start ,train_end = 0, floor(number_elements * train_ratio)
  val_start, val_end = ceil(number_elements * train_ratio), floor(number_elements * (train_ratio + val_ratio))
  test_start, test_end = ceil(number_elements * (train_ratio + val_ratio)), number_elements
  
  if train_end != val_start:
      val_start -= 1

  if val_end != test_start:
      test_start -= 1

  train = list_dicts[train_start:train_end]
  val =  list_dicts[val_start:val_end]
  test = list_dicts[test_start:test_end]
  
  return train, val, test

def get_coords_labels(list_dicts):
  """
  Takes a list of dicts as inputs and returns the GPS coordinates of each instance.
  """
  labels = np.empty((0, 2))
  for elem in list_dicts:
    labels = np.vstack((labels, np.array([elem['latitude'], elem['longitude']])))
  return labels

def get_timestamps(list_dicts):
    timestamps = np.empty((len(list_dicts), )).astype('str')
    for i, elem in enumerate(list_dicts):
        timestamps[i] = elem['gmtTimestamp']
    return timestamps

def my_std(vals):
  if vals.shape[0] == 0:
    return 0
  diff_squared = np.sum((vals - np.mean(vals)) ** 2)
  sqrd = np.sqrt(diff_squared / (vals.shape[0]) )
  return sqrd

def get_center_of_mass(num_grid_cells, y_nums, y_coords):
  center_of_mass = np.empty((num_grid_cells, 2))
  missing_flag, missing_idx = False, 0
  for i in range(num_grid_cells):
    idxs = np.where(y_nums==i)[0]

    if idxs.shape[0] == 0:
      if i > 0:
        center_of_mass[i] = center_of_mass[i-1]
      else:
        missing_flag = True
      continue
    samples = y_coords[idxs]
    center = np.mean(samples, axis=0)
    center_of_mass[i] = center
  
  if missing_flag:
    center_of_mass[missing_idx] = center_of_mass[missing_idx + 1]
  return center_of_mass

def get_prediction_coords(predictions_nums, center_of_mass):
  predictions_coords = np.empty((predictions_nums.shape[0], 2))
  
  for i, pred in enumerate(predictions_nums):
    predictions_coords[i] = center_of_mass[pred]
  
  return predictions_coords
    
def calculate_localization_errors(prediction_coords, true_coords):
  errors = np.empty(prediction_coords.shape[0])
  
  for i, (pred, true) in enumerate(zip(prediction_coords, true_coords)):
    errors[i] = get_distance(pred, true)
  
  return errors

def get_xy_coords(nums, rows, cols):
  xy_coords = []
  for num in nums:
    i, j = num / cols, num % cols
    xy_coords.append((i, j))
  return xy_coords

def get_manhattan_distance(x1, y1, x2, y2):
  return np.abs(x1 - x2) + np.abs(y1 - y2)

def calculate_accuracy(rows, cols, pred_nums, true_nums, rank=1):
  count_correct = 0
  count_all = pred_nums.shape[0]
  true_xy = get_xy_coords(true_nums, rows, cols)
  pred_xy = get_xy_coords(pred_nums, rows, cols)
  for xy1, xy2 in zip(true_xy, pred_xy):
    manhattan_dist = get_manhattan_distance(*xy1, *xy2)
    if manhattan_dist <= rank:
      count_correct += 1
  return count_correct/count_all

def get_ground_truth_error(data, center_of_mass):
  errors = np.zeros(data[0].shape[0])
  for i, (label, coords) in enumerate(zip(data[2], data[3])):
    label = int(label)
    lat,lon = coords[0], coords[1]
    center = center_of_mass[label]
    error = get_distance((lat, lon), center)
    errors[i] = error
  return errors


def load_data_from_csv(file_paths, keys, skip_rows, header_row_index=14, required_data=None):
  if type(file_paths) is not list:
    file_paths = list(file_paths)
    keys       = list(keys)
    skip_rows  = list(skip_rows)
    
  assert len(file_paths) == len(keys), 'Number of files must match number of keys'
  assert len(file_paths) == len(skip_rows), 'Number of files must match number of skip_rows'
  
  if required_data is None:
    required_data = ['gmtTimestamp', 'latitude', 'longitude', 'accuracy', 'GSM_CID', 'GSM_LAC', 'GSM_RSS', 'GSM_NCELLS']
  
  all_data = {}

  for key, path, skip_row in zip(keys, file_paths, skip_rows):
      data = []
      with open(path, newline='') as csvfile:
          reader = csv.DictReader(csvfile, fieldnames=[])
          for row in reader:
              data.append(row)

      data_keys = data[header_row_index][None]
      positions = [i for i in range(len(data_keys)) if data_keys[i] in required_data]

      list_dicts = []
      for i in tqdm(range(skip_row, len(data))):
          row = {}
          for j in range(len(positions)):
              row[required_data[j]] = data[i][None][positions[j]]

          list_dicts.append(row)
      all_data[key] = {}
      all_data[key]['original'] = list_dicts
      print(f'Finished reading data for key: {key}')
      
  return all_data


def get_preds_from_model(model, data, centers_of_mass):
  preds = model.predict(data)
  preds_nums = np.argmax(preds, axis=1)
  preds_coords = get_prediction_coords(preds_nums, centers_of_mass)
  
  return preds_nums, preds_coords

def normalize_labels(y_train, y_val, y_holdout=None):
  minimum = np.min(y_train, axis=0)
  range = np.max(y_train, axis=0) - minimum
  y_train = np.divide(y_train - minimum, range)
  y_val   = np.divide(y_val - minimum, range)
  if y_holdout is not None:
    y_holdout = np.divide(y_holdout - minimum, range)
  
  return minimum, range, y_train, y_val, y_holdout

def get_square_samples(X, num_channels=3):
    num_cell_towers = X.shape[1]
    X_square = np.empty((X.shape[0], num_channels, num_cell_towers,num_cell_towers))
    for i, sample in enumerate(X):
        tmps = np.empty((1,num_channels, num_cell_towers, num_cell_towers))
        tmp = tmps[0]
        
        tmp[0] = np.divide(sample.reshape(-1,1), sample.reshape(1,-1))
        if num_channels > 1:
            tmp[1] = sample.reshape(-1,1) - sample.reshape(1,-1)
            
        if num_channels > 2:
            tmp[2] = np.tile(sample, (num_cell_towers, 1))
      
        X_square[i] = tmps
        
    X_square = np.moveaxis(X_square, 1, -1)
    return (X_square, )

# Getting the data

In [4]:
!wget https://www.dropbox.com/s/wq3kzs5m4r3u2gl/Testbed2.rar
!unrar e Testbed2.rar

--2020-01-25 14:54:40--  https://www.dropbox.com/s/wq3kzs5m4r3u2gl/Testbed2.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wq3kzs5m4r3u2gl/Testbed2.rar [following]
--2020-01-25 14:54:41--  https://www.dropbox.com/s/raw/wq3kzs5m4r3u2gl/Testbed2.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5e019e6323077d38cb6e118f20.dl.dropboxusercontent.com/cd/0/inline/Aw056VFAMn8rdCdiDFSj3vMkbRmIIB_RVT6t7ZHtovaXao0SfTsa-eXyZ-TGJEm1RCNVvjx8Cxg7SezaEqV7Jyer1ZiDRekJxLKwfdSif7nlDf-s1Id-zYv_G9RMKpDHnO8/file# [following]
--2020-01-25 14:54:41--  https://uc5e019e6323077d38cb6e118f20.dl.dropboxusercontent.com/cd/0/inline/Aw056VFAMn8rdCdiDFSj3vMkbRmIIB_RVT6t7ZHtovaXao0SfTsa-eXyZ-TGJEm1RCNVvjx8Cxg7SezaEqV7Jyer1ZiDRekJxLKwfdSif7nlDf-s1I

# Data

In [5]:
file_path1 = 'Testbed2.csv'                                   # Path of the csv file
file_paths = [file_path1]


header_row_index = 14                                         # The index of the row that contains the columns' names
skip_rows = [200]                                             # The number of rows containing invalid data to skip
keys      = ['1']


all_data = load_data_from_csv(file_paths, 
                              keys,
                              skip_rows,
                              header_row_index=header_row_index)

100%|██████████| 89327/89327 [00:00<00:00, 415318.92it/s]


Finished reading data for key: 1


In [0]:
all_data['1']['changed']  = change_datatypes(all_data['1']['original'])

In [0]:
cell_indxs1 = get_cell_indxs(all_data['1']['changed'])
all_data['1']['combined'] = combine_cells(all_data['1']['changed'], cell_indxs=cell_indxs1, zero_value=-108)

In [8]:
grid_cell_size = 400              # Width/Height of each grid cell

lats, lons = get_min_max_coords(all_data['1']['combined'])
rows, cols = assign_cells_to_data_2(all_data['1']['combined'], (lats[0], lons[0], lats[1], lons[1]), grid_cell_size)
num_grid_cells = rows * cols
print(f'# of rows {rows}\t# of columns: {cols}\t# of grid cells: {num_grid_cells}')

3 5


In [0]:
# Split the data into train, validation and test sets.
np.random.seed(42)
all_data['1']['train'], all_data['1']['val'], all_data['1']['test'] = train_val_test_split(all_data['1']['combined'], 0.1, 0.2, True) # pass list of dicts, val_ratio, test_ratio, shuffle=True/False

# Create the data matrices for each set
all_data['1']['train'] = create_data_matrix(all_data['1']['train'], output_len=num_grid_cells)
all_data['1']['val']   = create_data_matrix(all_data['1']['val'], output_len=num_grid_cells)
all_data['1']['test']  = create_data_matrix(all_data['1']['test'], output_len=num_grid_cells)


In [10]:
# Get the center of mass of each cell in the grid
center_of_mass = get_center_of_mass(all_data['1']['train'][1].shape[1], all_data['1']['train'][2], all_data['1']['train'][3])

# Get the lowest achievable error on the train and validation set
trn_errors = get_ground_truth_error(all_data['1']['train'], center_of_mass)
val_errors = get_ground_truth_error(all_data['1']['val'], center_of_mass)

print('Lowest achievable median localization error using the classifcation approach')
print(f'Train median: {np.median(trn_errors):.4f}m\tVal median: {np.median(val_errors):.4f}m')

Train median: 112.48854291539558	Val median: 113.84045271866687


In [0]:
# Create the 3 channels
num_channels = 3
for key in ['1']:
  for split in ['train', 'val', 'test']:
    square_samples = get_square_samples(all_data[key][split][0], num_channels=num_channels)

    if len(all_data[key][split]) > 5:
      all_data[key][split] = all_data[key][split][:-1]

    all_data[key][split] += square_samples

# DeepCReg

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, Dropout, Activation, MaxPooling2D
from keras.optimizers import Adam, sgd, SGD
import keras.backend as K

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

## Callbacks

In [0]:
from keras.callbacks import Callback
from keras.callbacks import LearningRateScheduler

class RegressorsCallback(Callback):
  """
  Logs the localization error of the model on the training and validation sets
  during training and saves the models that achieve the lowest localization error.
  """
  def __init__(self, data,min_, range_, title, data_idx=-1):
    self.X        = data[data_idx]
    self.y_coords = data[3]
    
    self.min = min_
    self.range = range_
    
    self.title = title
    self.best_median = 1e5
    self.medians = []
    
  def on_train_begin(self, logs=None):
    self.best_err    = 1e5
    self.best_median = 1e5
    
    self.errors    = []
    self.medians = []

  
  def on_epoch_end(self, epoch, logs=None):
    pred_coords = np.multiply(self.model.predict(self.X), self.range) + self.min
    errors = calculate_localization_errors(pred_coords, self.y_coords)
    
    print(f'{self.title}')
    print(f'Localization Error: {np.min(errors):5f}\t{np.percentile(errors,25):5f}\t{np.median(errors):.5f}\t'\
      f'{np.percentile(errors,75):.5f}\t{np.max(errors):.5f}')
    
    self.medians.append(np.median(errors))
          
    if np.median(errors) < self.best_median:
      path = Path('.')
      for p in path.glob(f'best_{self.title}_median*'):
        p.unlink()
        
      self.best_median = np.median(errors)
      self.model.save(f'best_{self.title}_median-{self.best_median:.4f}_epoch-{epoch}', overwrite=True)

## Model

In [0]:
X_train, y_train, y_train_nums       = all_data['1']['train'][-1], all_data['1']['train'][3], all_data['1']['train'][2]
X_val, y_val, y_val_nums             = all_data['1']['val'][-1], all_data['1']['val'][3], all_data['1']['val'][2]
X_test, y_test, y_test_num           = all_data['1']['test'][-1], all_data['1']['test'][3], all_data['1']['test'][2]

In [0]:
min_, range_, y_train_normalized, y_val_normalized, y_test = normalize_labels(y_train, y_val, y_test)

In [16]:
keras.backend.set_image_data_format('channels_last')

num_cells = X_train.shape[1]
input_shape = (num_cells, num_cells, num_channels)

cnn_model = Sequential()
cnn_model.add(Conv2D(128, kernel_size=(X_train.shape[1], 5), activation='relu', input_shape=input_shape))
cnn_model.add(Conv2D(256, kernel_size=(1,5), activation='relu'))
cnn_model.add(Conv2D(512, kernel_size=(1,5), activation='relu'))
cnn_model.add(Flatten())
cnn_model.add(Dense(256))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(128))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(64))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(y_train.shape[1], activation='sigmoid'))
cnn_model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 54, 128)        111488    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 50, 256)        164096    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 46, 512)        655872    
_________________________________________________________________
flatten_1 (Flatten)          (None, 23552)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6029568   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)           

In [0]:
from keras.callbacks import ModelCheckpoint

def LRScheduler(epoch, current_lr):
  if epoch != 0 and epoch % 25 == 0:
    current_lr *= 0.1
  return current_lr

trn_clb = RegressorsCallback(all_data['1']['train'], min_, range_, title='train-reg')
val_clb = RegressorsCallback(all_data['1']['val'], min_, range_, title='validation-reg')
lr_sched = LearningRateScheduler(LRScheduler, verbose=1)
callbacks = [trn_clb, val_clb, lr_sched]

epochs=150
bs=64
initial_lr = 1e-4

In [18]:
optimizer = Adam(lr=initial_lr)
cnn_model.compile(optimizer, loss='mse', metrics=['mse'])
cnn_model.fit(X_train, y_train_normalized, batch_size=bs, epochs=epochs,
              validation_data=(X_val, y_val_normalized), callbacks=callbacks)




Train on 62528 samples, validate on 8933 samples
Epoch 1/150

Epoch 00001: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
62528/62528 [==============================] - 26s 422us/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 5.3536e-04 - val_mean_squared_error: 5.3536e-04
train-reg
Localization Error: 0.724858	16.511412	29.03612	45.94466	181.84740
validation-reg
Localization Error: 0.724858	16.588085	29.15687	46.57020	179.03626
Epoch 2/150

Epoch 00002: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
62528/62528 [==============================] - 17s 277us/step - loss: 2.7255e-04 - mean_squared_error: 2.7255e-04 - val_loss: 4.8839e-04 - val_mean_squared_error: 4.8839e-04
train-reg
Localization Error: 0.745221	16.523327	27.20729	42.81160	202.41293
validation-reg
Localization Error: 0.745221	16.506456	27.04118	43.37988	193.87932
Epoch 3/150

Epoch 00003: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
62

## Evaluation

In [0]:
pred_coords = np.multiply(cnn_model.predict(X_val), range_) + min_    
errors = calculate_localization_errors(pred_coords, all_data['1']['val'][3])

In [20]:
print(f'Best median localization error on the training set: {trn_clb.best_median:.4f}m')
print(f'Best median localization error on the validation set: {val_clb.best_median:.4f}m') 

Best median localization error on the training set: 2.7227m
Best median localization error on the validation set: 2.7430m


In [21]:
%ls       # Copy the name of the model that achieved the lowest median localization error on the validation set that has prefix 'best-validation-reg-*'

best_train-reg_median-2.7227_epoch-95       sample_data/  Testbed2.rar
best_validation-reg_median-2.7430_epoch-95  Testbed2.csv


In [24]:
cnn_model = keras.models.load_model('best_validation-reg_median-2.7430_epoch-95')  # paste the name you copied from the above cell
test_pred = np.multiply(cnn_model.predict(X_test), range_) + min_
test_errs = errors = calculate_localization_errors(test_pred, all_data['1']['test'][3])
print(f'Minimum: {np.min(test_errs):.4f}m, 25th Percentile: {np.percentile(test_errs, 25):.4f}m, Median: {np.median(test_errs):.4f}m, 75th: {np.percentile(test_errs, 75):.4f}m, Maximum: {np.max(test_errs):.4f}m')

Minimum: 0.0206m, 25th Percentile: 1.0733m, Median: 2.8167m, 75th: 6.2930m, Maximum: 163.8806m
